In [31]:
import json
import os

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
  license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

Saving spark_nlp_for_healthcare_spark_ocr_5493.json to spark_nlp_for_healthcare_spark_ocr_5493 (2).json


In [32]:
# This is only to setup PySpark and Spark NLP on Colab
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh

--2022-06-20 06:41:33--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2128 (2.1K) [text/plain]
Saving to: ‘jsl_colab_setup.sh.2’

jsl_colab_setup.sh. 100%[===================>]   2.08K  --.-KB/s    in 0s      

2022-06-20 06:41:33 (32.6 MB/s) - ‘jsl_colab_setup.sh.2’ saved [2128/2128]



In [33]:
# -p is for pyspark (by default 3.1.1)
!bash jsl_colab_setup.sh

setup Colab for PySpark 3.1.1 and Spark NLP 3.4.4
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [34]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [35]:
import json
import os

import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

spark

Spark NLP Version : 3.4.4
Spark NLP_JSL Version : 3.5.3


In [36]:
# if you want to start the session with custom params as in start function above
from pyspark.sql import SparkSession

def start(SECRET):
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:"+PUBLIC_VERSION) \
        .config("spark.jars", "https://pypi.johnsnowlabs.com/"+SECRET+"/spark-nlp-jsl-"+JSL_VERSION+".jar")
      
    return builder.getOrCreate()


In [37]:
from sparknlp.pretrained import PretrainedPipeline
finder_pipeline = PretrainedPipeline("ner_model_finder", "en", "clinical/models")

ner_model_finder download started this may take some time.
Approx size to download 148.6 MB
[OK!]


In [89]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
        
sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")
 
# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
    .setInputCols(["sentence","token"])\
    .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = NerDLModel.pretrained("deidentify_dl","en","clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner")\
     #decide if we want to return the tags in upper or lower case 

ner_converter = NerConverter()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter])


empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
deidentify_dl download started this may take some time.
Approximate size to download 14.1 MB
[OK!]


In [90]:
deidentification=DeIdentificationModel.pretrained("deidentify_rb","en","clinical/models")\
      .setInputCols(['sentence','token','ner_chunk'])\
      .setOutputCol('deidentified')\
      .setMode('mask')


deidentification=DeIdentificationModel.pretrained("deidentify_rb","en","clinical/models")\
      .setInputCols(['sentence','token','ner_chunk'])\
      .setOutputCol('deidentified')\
      .setMode('mask')

deidentify_rb download started this may take some time.
Approximate size to download 3.8 KB
[OK!]


In [55]:
osfuscation=DeIdentificationModel.pretrained("deidentify_large","en","clinical/models")\
      .setInputCols(['sentence','token','ner_chunk'])\
      .setOutputCol('deidentified')\
      .setMode('obfuscate')

deidentify_large download started this may take some time.
Approximate size to download 188.1 KB
[OK!]


In [91]:
nlpPipeline = Pipeline(stages=[documentAssembler, sentenceDetector, tokenizer, word_embeddings, clinical_ner, ner_converter,deidentification])
text = '''A . Record date : 2093-01-13 , David Hale , M.D . , Name : Hendrickson , Ora MR . # 7194334 Date : 01/13/93 PCP : Oliveira , 25 years-old , Record date : 2079-11-09 . Cocke County Baptist Hospital . 0295 Keats Street'''

model = nlpPipeline.fit(spark.createDataFrame([[text]]).toDF("text"))

    
output = model.transform(spark.createDataFrame([[text]]).toDF("text"))
# deid_text = deidentification.transform(result)

In [93]:
output.select('deidentified').show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|deidentified                                                                                                                                                                                                                                                                                                                                                                                      |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [79]:
deid_text[['sentence','deidentified']].show()

+--------------------+--------------------+
|            sentence|        deidentified|
+--------------------+--------------------+
|[{document, 0, 80...|[{document, 0, 76...|
+--------------------+--------------------+



In [86]:
deid_text

DataFrame[text: string, document: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, sentence: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, token: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, embeddings: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, ner: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, ner_chunk: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, deidentified: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>]

In [95]:
#  Downloading sample datasets.
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/mt_samples.csv

In [99]:
import pandas as pd

# mt_samples_df = spark.read.csv("mt_samples.csv", header=True)

from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [103]:
ls

 jsl_colab_setup.sh           spark-3.1.1-bin-hadoop2.7.tgz
 jsl_colab_setup.sh.1         spark-3.1.1-bin-hadoop2.7.tgz.1
 jsl_colab_setup.sh.2         spark-3.1.1-bin-hadoop2.7.tgz.2
 mt_samples.csv              'spark_nlp_for_healthcare_spark_ocr_5493 (1).json'
 sample_data/                'spark_nlp_for_healthcare_spark_ocr_5493 (2).json'
 spark-3.1.1-bin-hadoop2.7/   spark_nlp_for_healthcare_spark_ocr_5493.json


In [112]:
data_file = "/content/mt_samples.csv"
data_df = spark.read.csv(data_file)

In [113]:
data_df=data_df.withColumnRenamed("_c0","text")

In [114]:
data_df.show()

+--------------------+
|                text|
+--------------------+
|                text|
|Sample Type / Med...|
|Hematology - Onco...|
|        Sample Name:|
|Discharge Summary...|
|        Description:|
|        Mesothelioma|
|(Medical Transcri...|
|PRINCIPAL DIAGNOSIS:|
|       Mesothelioma.|
|SECONDARY DIAGNOSES:|
|    Pleural effusion|
|          PROCEDURES|
|     1. On August 24|
|     2. On August 20|
|     3. On August 31|
|HISTORY AND PHYSI...|
|The patient is a ...|
|PAST MEDICAL HISTORY|
|  1. Pericardectomy.|
+--------------------+
only showing top 20 rows



In [116]:
model = nlpPipeline.fit(data_df)

    
output = model.transform(data_df)

In [120]:
output.select('deidentified').show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|deidentified                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
+---------------------------

In [119]:
output.select('text').show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                                                                                      |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                            